In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re

import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
os.makedirs("figures", exist_ok=True)

In [ ]:
# equivalent = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2e")
carbon = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2")
methane = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CH4")
nitrous = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="N2O")

In [ ]:
year = "2018"
for missing in ["NO", "NE", "NO,NA"]:
    carbon[year][carbon[year] == missing] = 0
    nitrous[year][nitrous[year] == missing] = 0
    methane[year][methane[year] == missing] = 0
    
    carbon.fillna(0.0, inplace=True)
    nitrous.fillna(0.0, inplace=True)
    methane.fillna(0.0, inplace=True)

In [ ]:
with open("data/structure_pretty.json", "r") as f:
    structure = json.load(f)

In [ ]:
with open("data/labels.json", "r") as f:
    label_dict = json.load(f)

In [ ]:
carbon_scaling = 1

In [ ]:
sources = []
targets = []
value = []
labels = []


def add_items(source, target):
    if source not in labels:
        labels.append(source)
    if target not in labels:
        labels.append(target)
    sources.append(labels.index(source))
    targets.append(labels.index(target))


for start in list(structure.keys()):
    s = structure[start]

    if type(s) is dict:

        for key, val in s.items():
            add_items(start, key)
            value.append(
                carbon.loc[key, year] * carbon_scaling +  nitrous.loc[key, year] + methane.loc[key, year]
            )

            if not val:
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(key, t)
                    if t == "CO2":
                        value.append(carbon.loc[key, year])
                    elif t == "N2O":
                        value.append(nitrous.loc[key, year])
                    elif t == "CH4":
                        value.append(methane.loc[key, year])

            for v in val:
                value.append(
                    carbon.loc[v, year] * carbon_scaling +  nitrous.loc[v, year] + methane.loc[v, year]
                )
                add_items(key, v)

                # Final level
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(v, t)
                    if t == "CO2":
                        value.append(carbon.loc[v, year] * carbon_scaling)
                    elif t == "N2O":
                        value.append(nitrous.loc[v, year])
                    elif t == "CH4":
                        value.append(methane.loc[v, year])

    else:
        for val in s:
            add_items(start, val)
            value.append(
                abs(carbon.loc[val, year]) * carbon_scaling +  nitrous.loc[val, year] + methane.loc[val, year]
            )

            # Final level
            for t in ["CO2", "N2O", "CH4"]:
                add_items(val, t)
                if t == "CO2":
                    value.append(abs(carbon.loc[val, year]) * carbon_scaling)
                elif t == "N2O":
                    value.append(nitrous.loc[val, year])
                elif t == "CH4":
                    value.append(methane.loc[val, year])


In [ ]:
flows = pd.DataFrame(data={
    'source_id': sources,
    'target_id': targets,
    'source': [labels[s] for s in sources],
    'target': [labels[t] for t in targets],
    'value': value
})

In [ ]:
with open("data/colors.json", "r") as f:
    colors = json.load(f)

In [ ]:
flows["link_color"] = [colors[s] for s in flows.source]

In [ ]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".1f",
    arrangement="perpendicular",
    orientation="h",
    node=dict(
        pad=8,
        thickness=10,
        line=dict(color="black", width=0.5),
        label=[label_dict[l] for l in labels],
        color=[re.sub(', \.\d', '', colors[l]).replace("rgba", "rgb") for l in labels]
    ),
    link=dict(
        source=flows["source_id"],
        target=flows["target_id"],
        value=flows["value"],
        color=flows["link_color"],
    ))])

fig.update_layout(title_text=f"Danish GHG Emissions in {year}", font_size=10)
fig.show(config={"responsive": True, "displayModeBar": False})

In [ ]:
# fig.write_html("figures/sankey.html", config={"responsive": True, "displayModeBar": False})

In [ ]:
fig.write_image("figures/sankey-medium.svg", width=1920, height=1080, scale=1)
fig.write_image("figures/sankey-medium.png", width=1920, height=1080, scale=1)

In [ ]:
import plotly.graph_objects as go
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

In [ ]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 0,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()